## Some important concepts of keras
1. We can use custom losses and custom metrics see : https://keras.io/guides/training_with_built_in_methods/#the-compile-method-specifying-a-loss-metrics-and-an-optimizer

2. We can use class weights and sample weights for imbalaced classification see : above link


In [1]:
#from keras.initializers import Constant
#embeddings_initializer=Constant(embedding_matrix)
#We can give class_weight arg to fit() a dic to balance classes
#from keras.utils.np_utils import to_categorical convert labels into arrays of integers(better then one-hot/label encoder)


In [2]:
pip install --upgrade jupyter_http_over_ws>=0.0.7 
#  jupyter serverextension enable --py jupyter_http_over_ws

SyntaxError: invalid syntax (<ipython-input-2-eb69f7434fa0>, line 1)

In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
import re
pd.set_option('max_colwidth', 1000)
import numpy as np
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer as wnl
import sklearn
import joblib

C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os
PATH = os.path.join("C:/BEPEC Python Material/NLP,Naive Baise,SVM/","NB.csv")
print(PATH)
df = pd.read_csv(PATH , encoding = 'ISO-8859-1')
df.head()

C:/BEPEC Python Material/NLP,Naive Baise,SVM/NB.csv


,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+"
4,spam,okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm


In [3]:
df['type'].replace(to_replace='ham', value='non-spam', inplace=True)
df.rename(columns={'text':'Text'}, inplace=True)
df['type'].replace(to_replace=['non-spam','spam'], value=[0,1], inplace=True)
df.head(3)

,type,Text
0,0,Hope you are having a good week. Just checking in
1,0,K..give back my thanks.
2,0,Am also doing in cbe only. But have to pay.


In [4]:
def clean_text(text):  
    lower = text.lower()
    words = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", lower )  
    #pattren = r"[^\w\s\d]"
    #words = re.sub(pattren, "", lower)
    words2 = words.split()
    #print(words2)
    final_words =  [wnl().lemmatize(word , pos = 'v') for word in words2 if word not in stopwords.words('english')]
    final_words = ' '.join(final_words)
    return(final_words)
    
df['Cleaned-text'] = df['Text'].apply(clean_text)
df.head()

,type,Text,Cleaned-text
0,0,Hope you are having a good week. Just checking in,hope good week check
1,0,K..give back my thanks.,kgive back thank
2,0,Am also doing in cbe only. But have to pay.,also cbe pay
3,1,"complimentary 4 STAR Ibiza Holiday or å£10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+",complimentary 4 star ibiza holiday 10000 cash need urgent collection 09066364349 landline lose box434sk38wp150ppm18
4,1,okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm,okmail dear dave final notice collect 4 tenerife holiday 5000 cash award call 09061743806 landline tcs sae box326 cw25wx 150ppm


In [5]:
import sklearn
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(df['Cleaned-text'],df['type'],test_size = 0.1,random_state = 42,shuffle= True,
                                                      stratify = df['type'])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test.value_counts())

(5003,)
(556,)
(5003,)
(556,)
0    481
1     75
Name: type, dtype: int64


In [6]:
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [7]:
def create_corpus(df):
    corpus=[]
    for text in tqdm(df):
        words=[word for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [8]:
train_corpus = create_corpus(X_train)
test_corpus = create_corpus(X_test)
test_corpus[:5]

100%|██████████████████████████████████████████████████████████████████████████████| 556/556 [00:00<00:00, 5454.01it/s]


[['happen', 'dear', 'silent', 'tense'],
 ['reverse', 'cheat', 'mathematics'],
 ['ever', 'one', 'foot'],
 ['havent',
  'mus',
  'ask',
  'u',
  '1st',
  'wat',
  'meet',
  '4',
  'lunch',
  'den',
  'u',
  'n',
  'meet',
  'already',
  'lor',
  'u',
  'wan',
  '2',
  'go',
  'ask',
  'da',
  'ge',
  '1st',
  'confirm',
  'w',
  'asap'],
 ['hows', 'champ', 'leave', 'glasgow']]

In [10]:
tokenizer_obj = Tokenizer(num_words = 4000, # max number of words to be taken
                         filters =  '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t0-9')
tokenizer_obj.fit_on_texts(train_corpus) # methods `texts_to_sequences` or `texts_to_matrix`.
train_sequences=tokenizer_obj.texts_to_sequences(train_corpus) # Transforms each text in texts to a sequence of integers.
test_sequences=tokenizer_obj.texts_to_sequences(test_corpus)

test_sequences[:5]

[[187, 70, 1375],
 [3859, 1911, 3860],
 [285, 30],
 [181,
  48,
  1,
  276,
  80,
  46,
  10,
  208,
  342,
  1,
  44,
  46,
  87,
  36,
  1,
  88,
  5,
  4,
  48,
  52,
  1441,
  276,
  618,
  673,
  815],
 [224, 64]]

In [11]:
train_pad=pad_sequences(train_sequences,maxlen=50,truncating='post',padding='pre') # lists to array form
test_pad=pad_sequences(test_sequences,maxlen=50,truncating='post',padding='pre')

print(len(train_corpus))
print(train_pad.shape)
train_pad

5003
(5003, 50)


array([[   0,    0,    0, ...,   31,  533,  973],
       [   0,    0,    0, ...,  151,  697,  226],
       [   0,    0,    0, ...,  138,   94, 3506],
       ...,
       [   0,    0,    0, ...,   19,  757,  295],
       [   0,    0,    0, ...,  197,  957,  168],
       [   0,    0,    0, ...,    0,  494, 3156]])

In [12]:
word_index=tokenizer_obj.word_index
lenght = len(word_index)+1
print('Number of unique words:',lenght)

Number of unique words: 7928


## Model building

1. Without drop and with dropout both will be examined and a 3rd architecture

In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import Constant

C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
DIM = 100
MAX_LEN = 50
model = Sequential()
lenght = 7928
## Embedding layer
model.add(Embedding(lenght , DIM , input_length = MAX_LEN , trainable=True))
#model.compile('adam' , 'mse')

## LSTM layer
model.add(Bidirectional(LSTM(128,return_sequences=True)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
accuracy_metric = tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5)
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=accuracy_metric) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
#mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

#Model summary
model.summary()

<IPython.core.display.Javascript object>

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           792800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           234496    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,043,809
Trainable params: 1,043,809
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(np.array(train_pad),np.array(y_train),batch_size=25,epochs=10,
                    validation_data=(np.array(test_pad),np.array(y_test)),verbose=1,callbacks=[es],
                    )

Epoch 1/10
201/201 [==============================] - 22s 88ms/step - loss: 0.3921 - binary_accuracy: 0.8721 - val_loss: 0.0690 - val_binary_accuracy: 0.9802
Epoch 2/10
201/201 [==============================] - 18s 89ms/step - loss: 0.0480 - binary_accuracy: 0.9870 - val_loss: 0.0668 - val_binary_accuracy: 0.9802
Epoch 3/10
201/201 [==============================] - 17s 86ms/step - loss: 0.0203 - binary_accuracy: 0.9937 - val_loss: 0.0543 - val_binary_accuracy: 0.9820
Epoch 4/10
201/201 [==============================] - 19s 94ms/step - loss: 0.0082 - binary_accuracy: 0.9988 - val_loss: 0.0792 - val_binary_accuracy: 0.9748
Epoch 5/10
201/201 [==============================] - 19s 96ms/step - loss: 0.0047 - binary_accuracy: 0.9993 - val_loss: 0.1045 - val_binary_accuracy: 0.9802
Epoch 6/10
201/201 [==============================] - 17s 83ms/step - loss: 0.0014 - binary_accuracy: 0.9996 - val_loss: 0.0722 - val_binary_accuracy: 0.9802
Epoch 00006: early stopping


In [20]:
expected = y_test
y_pred = model.predict_classes(np.array(test_pad))
from sklearn import metrics
print(metrics.classification_report(expected, y_pred))
print(metrics.confusion_matrix(expected, y_pred))

C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ValueError: Classification metrics can't handle a mix of binary and unknown targets

## Model with dropouts 
1. ACtually dropouts are good in LSTM layer so always use them !!

In [40]:
## Model_2
model = Sequential()

## Embedding layer
model.add(Embedding(lenght , DIM , input_length = MAX_LEN , trainable=True))
model.compile('adam' , 'mse')

## LSTM layer
model.add(LSTM(128,return_sequences=True , dropout=0.2 , recurrent_dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
accuracy_metric = tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5)
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=accuracy_metric) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
#mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)
#checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)

#Model summary
model.summary()

<IPython.core.display.Javascript object>

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 50, 100)           792800    
_________________________________________________________________
lstm_12 (LSTM)               (None, 50, 128)           117248    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 918,369
Trainable params: 918,369
Non-trainable params: 0
_________________________________________________________________


In [41]:
history = model.fit(np.array(train_pad),np.array(y_train),batch_size=25,epochs=10,
                    validation_data=(np.array(test_pad),np.array(y_test)),verbose=1,callbacks=[es],
                    )

Epoch 1/10


KeyboardInterrupt: 

In [16]:
expected = y_test
y_pred = model.predict_classes(np.array(test_pad))

print(metrics.classification_report(expected, y_pred))
print(metrics.confusion_matrix(expected, y_pred))

C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       481
           1       0.97      0.87      0.92        75

    accuracy                           0.98       556
   macro avg       0.97      0.93      0.95       556
weighted avg       0.98      0.98      0.98       556

[[479   2]
 [ 10  65]]


## 3rd architecture

In [50]:
## Model_3
model = Sequential()

## Embedding layer
model.add(Embedding(lenght , DIM , input_length = MAX_LEN , trainable=True))
#model.compile('adam' , 'mse')

## LSTM layer
model.add(LSTM(128,return_sequences=True , dropout=0.2 , recurrent_dropout=0.2))
model.add(GlobalMaxPooling1D())
## Dense layers
#model.add(TimeDistributed(Dense(lenght)))
#model.add(Activation('softmax'))
model.add(Dense(1 , activation = 'sigmoid'))
#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=accuracy_metric) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  

#Model summary
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 50, 100)           792800    
_________________________________________________________________
lstm_18 (LSTM)               (None, 50, 128)           117248    
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 910,177
Trainable params: 910,177
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model.fit(np.array(train_pad),np.array(y_train),batch_size=25,epochs=10,
                    validation_data=(np.array(test_pad),np.array(y_test)),verbose=1,callbacks=[es],
                    )

Epoch 1/10
201/201 [==============================] - 36s 153ms/step - loss: 0.4045 - binary_accuracy: 0.8572 - val_loss: 0.1167 - val_binary_accuracy: 0.9658
Epoch 2/10
201/201 [==============================] - 33s 165ms/step - loss: 0.0578 - binary_accuracy: 0.9852 - val_loss: 0.0587 - val_binary_accuracy: 0.9820
Epoch 3/10
201/201 [==============================] - 33s 166ms/step - loss: 0.0261 - binary_accuracy: 0.9946 - val_loss: 0.0627 - val_binary_accuracy: 0.9820
Epoch 4/10
201/201 [==============================] - 29s 145ms/step - loss: 0.0185 - binary_accuracy: 0.9964 - val_loss: 0.0652 - val_binary_accuracy: 0.9802
Epoch 5/10
201/201 [==============================] - 29s 144ms/step - loss: 0.0084 - binary_accuracy: 0.9984 - val_loss: 0.0708 - val_binary_accuracy: 0.9784
Epoch 00005: early stopping


In [46]:
expected = y_test
y_pred = model.predict_classes(np.array(test_pad))

print(metrics.classification_report(expected, y_pred))
print(metrics.confusion_matrix(expected, y_pred))

C:\Users\Ali\.conda\envs\tensor_ali\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       481
           1       0.93      0.91      0.92        75

    accuracy                           0.98       556
   macro avg       0.96      0.95      0.95       556
weighted avg       0.98      0.98      0.98       556

[[476   5]
 [  7  68]]


## Susan li

In [5]:
model = Sequential()
model.add(Embedding(lenght, DIM, input_length= MAX_LEN))
model.add(SpatialDropout1D(0.2)) # performs variational dropouts in NLP model
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=accuracy_metric) 

epochs = 5
batch_size = 64
model.summary()
#history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           792800    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 50, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 873,301
Trainable params: 873,301
Non-trainable params: 0
_________________________________________________________________
